# Funcomics

Here, I should run all the fun stuff & save as mudata for next chapter

## Setup Env

In [3]:
import os, fnmatch

import numpy as np
import pandas as pd

import scanpy as sc
import squidpy as sq

In [4]:
import liana as li
from liana.method.sp import lr_basis, basis

In [5]:
import decoupler as dc
import seaborn as sns

In [23]:
data_dir = "/home/dbdimitrov/Repos/liana2_manuscript/data/kidney_injury/visium/"
slides = fnmatch.filter(os.listdir(data_dir + "deconv/"), '*.h5ad')

In [29]:
adatas = dict()

for slide in slides:
    sample = slide[:-5].split("_")[0]
    adatas[sample] = sc.read_h5ad(os.path.join(data_dir, "deconv", slide))

In [30]:
adatas

{'f6wks': AnnData object with n_obs × n_vars = 1514 × 16050
     obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'leiden'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
     uns: 'log1p', 'spatial'
     obsm: 'spatial', 'tangram_ct_pred'
     layers: 'counts',
 'f4hr': AnnData object with n_obs × n_vars = 1847 × 13911
     obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'leiden'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
     uns: 'log1p', 's

#### CytoSig

In [62]:
# load cytosig signatures
cytosig = pd.read_csv("https://raw.githubusercontent.com/data2intelligence/CytoSig/master/CytoSig/signature.centroid", sep="\t")
cytosig = cytosig.unstack().reset_index().rename(columns={'level_0':'source', 'level_1':'target', 0:'weight'})
# translate to murine
cytosig = dc.translate_net(net=cytosig, target_organism="mouse", columns=["target"])

#### PROGENy

In [65]:
progeny = dc.get_progeny(organism='Mus musculus', top=500)

#### Dorothea

In [68]:
dorothea = dc.get_dorothea(organism='mouse')

## Run on all samples

In [72]:
for sample in adatas.keys():
    adata = adatas[sample]
    
    # Run CytoSig
    dc.run_mlm(adata, cytosig, use_raw=False)
    adata.obsm['cytosig_estimate'] = adata.obsm['mlm_estimate']
    
    # Run PROGENy
    dc.run_mlm(adata, progeny, use_raw=False)
    adata.obsm['progeny_estimate'] = adata.obsm['mlm_estimate']
    
    # Run Dorothea
    dc.run_mlm(adata, progeny, use_raw=False)
    adata.obsm['dorothea_estimate'] = adata.obsm['mlm_estimate']
    
    del adata.obsm['mlm_estimate']
    del adata.obsm['mlm_pvals']

## Convert to mdatas & save

In [73]:
adatas

{'f6wks': AnnData object with n_obs × n_vars = 1514 × 16050
     obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'leiden'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
     uns: 'log1p', 'spatial'
     obsm: 'spatial', 'tangram_ct_pred', 'progeny_estimate', 'cytosig_estimate', 'dorothea_estimate'
     layers: 'counts',
 'f4hr': AnnData object with n_obs × n_vars = 1847 × 13911
     obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'leiden'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 